In [6]:
import sirf.STIR as STIR
import brainweb
from tqdm.auto import tqdm
import numpy
import os

In [7]:
dir = os.path.dirname("minimal.ipynb")
msg = STIR.MessageRedirector(dir+"/info.txt", dir+"/warnings.txt", dir+"/error.txt")

In [8]:
fname, url= sorted(brainweb.utils.LINKS.items())[0]
files = brainweb.get_file(fname, url, ".")
data = brainweb.load_file(fname)

brainweb.seed(1337)

In [9]:
for f in tqdm([fname], desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(f, petNoise=1, t1Noise=0.75, t2Noise=0.75, petSigma=1, t1Sigma=1, t2Sigma=1)
fdg_arr = vol['PET']

mMR ground truths:   0%|          | 0/1 [00:00<?, ?subject/s]

In [10]:
# Select central slice
central_slice = fdg_arr.shape[0]//2
fdg_arr = fdg_arr[central_slice, :, :]

# Select a central ROI with 120x120
idim = [120,120]
offset = (numpy.array(fdg_arr.shape) - numpy.array(idim)) // 2
fdg_arr = fdg_arr[offset[0]:offset[0]+idim[0], offset[1]:offset[1]+idim[1]]

# Now we make sure our image is of shape (1, 120, 120) 
fdg_arr = fdg_arr[numpy.newaxis,...]

In [11]:
templ_sino = STIR.AcquisitionData("template_sinogram.hs") # create an empty sinogram using a template

In [12]:
im = STIR.ImageData(templ_sino)
dim = fdg_arr.shape
voxel_size=im.voxel_sizes()
im.initialise(dim,(voxel_size[0]*2, voxel_size[1], voxel_size[2]))
fdg = im.clone().fill(fdg_arr)

In [13]:
acq_model_matrix = STIR.SPECTUBMatrix()
acq_model_matrix.set_resolution_model(0,0,full_3D=False)
am = STIR.AcquisitionModelUsingMatrix(acq_model_matrix)
am.set_up(templ_sino, fdg)

In [14]:
sino = am.forward(fdg)

no additive term added
no unnormalisation applied
no background term added


In [15]:
bp = am.backward(sino)

backprojecting...ok


In [16]:
sino = am.forward(bp)

no additive term added
no unnormalisation applied
no background term added


In [17]:
bp = am.backward(sino)

backprojecting...ok


In [18]:
sino = am.forward(bp)

no additive term added
no unnormalisation applied
no background term added


In [19]:
sino = am.forward(fdg)
bp = am.backward(sino)
sino = am.forward(bp)
bp = am.backward(sino)

no additive term added
no unnormalisation applied
no background term added
backprojecting...ok
no additive term added
no unnormalisation applied
no background term added
backprojecting...ok
